# 베트남 발화자 음성데이터를 활용한 whisper model fine tuning
### 작성자 : 3조 데이터 팀 - 권영후, 양채연, 정성욱

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [ ]:
!pip install accelerate -U

## 작업 환경 준비

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Aug  3 08:50:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   54C    P8    27W / 250W |    421MiB /  7979MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Whisper model 을 fine tuning하기 위해 사용할 Python 패키지
학습 데이터를 다운로드하고 준비하기 위해 datasets을 사용하였다.
transformers를 사용하여 Whisper 모델을 로드하고 학습시켰다.
오디오 파일을 사전 처리하기 위해 soundfile 을 사용하였고, 성능을 평가하기 위해 evaluate와 jiwer를 사용하였다.
마지막으로 데모를 만들기 위해 gradio를 사용하였다.

In [5]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-34iu3x4m
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-34iu3x4m
  Resolved https://github.com/huggingface/transformers to commit 2be8a9098e06262bdd5c16b5e8a70f145df88e96
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7605409 sha256=025c5270128f630a0173beaea340b2d50c50426198da8e5a06457f48c2e88510
  Stored in directory: /tmp/pip-ephem-wheel-cache-3n7t4235/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.1 

## hugging face hub에서 모델을 업로드 및 다운로드 하기 위해 notebook_login 사용

In [6]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset load
- ai hub에서 받은 로컬의 dataset을 사용하기 위한 작업
    - 아래의 csv 파일은 음성데이터에 대한 **라벨 데이터**와 음성파일에 대한 **경로**가 포함되어 있습니다.

In [ ]:
import pandas as pd
# import os
# # label_path = os.path.join(os.getcwd(),'kr_viet','label_data')
# # label_name_list = os.listdir(label_path)
# # for label in label_name_list:
# #     print(label)

# df1 = pd.read_csv('/content/한국일반.csv',
#                  encoding = 'cp949')['한국일반'].to_list()

# df2 = pd.read_csv('/content/한국생활I.csv',
#                  encoding = 'cp949')['한국생활I'].to_list()

# df3 = pd.read_csv('/content/한국생활II.csv',
#                  encoding = 'cp949')['한국생활II'].to_list()

# df4 = pd.read_csv('/content/한국문화I.csv',
#                  encoding = 'cp949')['한국문화I'].to_list()

# df5 = pd.read_csv('/content/한국문화II.csv',
#                  encoding = 'cp949')['한국문화II'].to_list()
# df_lst = [df1,df2,df3,df4,df5]

## Dataset을 정규화 하기 위한 작업
- Hugging Face의 DatasetDict 자료형을 활용해 음성데이터에 대한 정보를 저장합니다.
    - sampling rate는 16000으로 통일
    - 'audio': 음성파일 --> numpy array
    - 'sentence': 음성파일의 라벨데이터
- train/test를 8:2 비율로 나눕니다.

In [7]:
from datasets import DatasetDict, concatenate_datasets,load_dataset, Dataset,Audio,concatenate_datasets

# AUDIO_COLUMN_NAME = "audio"
# TEXT_COLUMN_NAME = "sentence"


# def normalize_dataset(ds, audio_column_name=None, text_column_name=None):
#     if audio_column_name is not None and audio_column_name != AUDIO_COLUMN_NAME:
#         ds = ds.rename_column(audio_column_name, AUDIO_COLUMN_NAME)
#     if text_column_name is not None and text_column_name != TEXT_COLUMN_NAME:
#         ds = ds.rename_column(text_column_name, TEXT_COLUMN_NAME)
#     # resample to the same sampling rate
#     ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
#     # normalise columns to ["audio", "sentence"]
#     ds = ds.remove_columns(set(ds.features.keys()) - set([AUDIO_COLUMN_NAME, TEXT_COLUMN_NAME]))
#     return ds

In [ ]:
# ds_norm = DatasetDict()


# dataset_lst = []
# # path = os.path.join(os.getcwd(),'kr_viet')
# dataset_path = '/content/drive/MyDrive/audio_data/TS_Viet/1.Kor_general'
# dataset1 = load_dataset(dataset_path,"ko")
# dataset1['train'] = dataset1['train'].add_column("sentence",df1)
# dataset_lst.append(dataset1['train'])

# print(dataset1['train'])
# print(dataset_lst)

# dataset_path = '/content/drive/MyDrive/audio_data/TS_Viet/2. 한국생활I'
# dataset2 = load_dataset(dataset_path,"ko")
# dataset2['train'] = dataset2['train'].add_column("sentence",df2)
# dataset_lst.append(dataset2['train'])


# dataset_path = '/content/drive/MyDrive/audio_data/TS_Viet/3. 한국생활II'
# dataset3 = load_dataset(dataset_path,"ko")
# dataset3['train'] = dataset3['train'].add_column("sentence",df3)
# dataset_lst.append(dataset3['train'])

# dataset_path = '/content/drive/MyDrive/audio_data/TS_Viet/4. 한국문화I'
# dataset4 = load_dataset(dataset_path,"ko")
# dataset4['train'] = dataset4['train'].add_column("sentence",df4)
# dataset_lst.append(dataset4['train'])


# dataset_path = '/content/drive/MyDrive/audio_data/TS_Viet/5. 한국문화II'
# dataset5 = load_dataset(dataset_path,"ko")
# dataset5['train'] = dataset5['train'].add_column("sentence",df5)
# dataset_lst.append(dataset5['train'])

# dataset = concatenate_datasets(dataset_lst)



# dataset_split = dataset.train_test_split(test_size=0.2,
#                                                   shuffle = True, seed = 200)
# print(dataset_split)


# ds_norm['train'] = normalize_dataset(dataset_split["train"],
#                                      AUDIO_COLUMN_NAME,TEXT_COLUMN_NAME)
# ds_norm['test'] = normalize_dataset(dataset_split["test"],
#                                      AUDIO_COLUMN_NAME,TEXT_COLUMN_NAME)
# print(ds_norm)
# # # raw_datasets['train'] = ds_train
# # # raw_datasets["train"] = raw_datasets["train"].shuffle(seed=10)
# # # raw_datasets["test"] = normalize_dataset(datasets["test"],AUDIO_COLUMN_NAME,TEXT_COLUMN_NAME)

### DatasetDict를 디스크에 저장합니다.

In [ ]:
# ds_norm.save_to_disk("/content/drive/MyDrive/bridze_data")

### 위의 전처리 과정을 거친 데이터셋을 load
- 경로는 수정이 필요합니다.
- train데이터 12만개
- test데이터 3만개

In [9]:
from datasets import DatasetDict
# 이후에 불러올 때는 load_from_disk 메서드를 사용합니다.
loaded_dataset_dict = DatasetDict.load_from_disk("/content/drive/MyDrive/bridze_data") # 경로 수정 필요

# 불러온 데이터셋을 확인하거나 처리할 수 있습니다.
print(loaded_dataset_dict["train"])
print(loaded_dataset_dict["test"])

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 124105
})
Dataset({
    features: ['audio', 'sentence'],
    num_rows: 31027
})


## Feature extrator, tokenizer,sequence-to-sequence mapping 준비작업

### WhisperFeatureExtractor 준비

In [11]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

### WhisperTokenizer 준비
한국어의 특성에 맞게 parameter 수정

In [12]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", padding_side = 'left',
                                             language="Korean",
                                             task="transcribe",
                                            clean_up_tokenization_spaces  = True,
                                            is_split_into_words = True)
tokenizer

WhisperTokenizer(name_or_path='openai/whisper-base', vocab_size=50258, model_max_length=448, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<|hi|>', '<|fi|>', '<|vi|>', '<|he|>', '<|uk|>', '<|el|>', '<|ms|>', '<|cs|>', '<|ro|>', '<|da|>', '<|hu|>', '<|ta|>', '<|no|>', '<|th|>', '<|ur|>', '<|hr|>', '<|bg|>', '<|lt|>', '<|la|>', '<|mi|>', '<|ml|>', '<|cy|>', '<|sk|>', '<|te|>', 

### WhisperProcessor를 만들기 위해 Whisper feature extrator와tokenizer를 결합

In [13]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")
processor

WhisperProcessor:
- feature_extractor: WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: WhisperTokenizer(name_or_path='openai/whisper-base', vocab_size=50258, model_max_length=448, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscri

### 데이터 준비

In [14]:
def prepare_dataset(batch,feature_extractor = feature_extractor,tokenizer = tokenizer):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

- 데이터셋의 .map 메서드를 사용하여 모든 학습 예제에 데이터 준비 함수를 적용.
- 인자 num_proc은 몇 개의 CPU 코어를 사용할지를 지정한다. num_proc을 1보다 크게 설정하면 병렬 처리가 활성화됨.
- 병렬 처리로 .map 메서드가 정지하는 경우, num_proc=1로 설정하고 데이터셋을 순차적으로 처리한다.

In [ ]:
bridze_map = loaded_dataset_dict.map(prepare_dataset, remove_columns=loaded_dataset_dict.column_names["train"], num_proc=2)

## Training과 평가

데이터를 준비했으므로, 훈련 파이프라인으로 진입할 준비가 되었음.

- 데이터 콜레이터 정의: 데이터 콜레이터는 우리의 전처리된 데이터를 가져와서 모델에 사용할 준비가 된 PyTorch 텐서로 준비함.

- 평가 지표 정의: 평가 과정에서는 모델을 단어 오류율 (CER) 지표를 사용하여 평가하고자 함.

- 사전 훈련된 체크포인트 로드: 사전 훈련된 체크포인트를 로드하고 훈련에 맞게 tuning.

- 훈련 구성 정의:  Trainer가 훈련 스케줄을 정의하는 데 사용됨

- 모델을 세밀하게 조정한 후, 테스트 데이터에서 평가하여 음성 인식이 제대로 이루어졌는지 확인함. 이 평가는 훈련된 모델이 한국어 음성을 정확하게 인식했는지 확인하는 데 사용됨

### Data Collator 정의

In [ ]:
from datasets import DatasetDict, concatenate_datasets, load_dataset, Dataset, Audio, concatenate_datasets
import os
print(os.getcwd())

### mapping 후의 DatasetDict 파일도 용량이 커서 load하여 사용하는 방식을 사용했습니다.
- mapping 전의 DatsetDict와 feature의 차이점을 확인할 수 있습니다.
  - 'audio' --> 'input_features'
  - 'sentences' --> 'labels'

In [ ]:
bridze_data = DatasetDict.load_from_disk("/Documents/whisper_tuning/bridze_map")

In [ ]:
bridze_data

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 124105
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 31027
    })
})

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

정의한 데이터 콜레이터를 초기화

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

- 한국어 음성 인식 모델의 평가 지표로 문자 오류율 (CER)을 사용할 것이다.
- CER은 많은 음성 인식 작업에서 평가 지표로 널리 사용되며 모델이 텍스트를 얼마나 정확하게 인식하는지를 측정한다.
- CER이 낮을수록 모델의 성능이 더 우수하다.
- CER을 사용하여 모델의 정확도를 측정하고 결과를 평가할 것이다.

In [ ]:
import evaluate

metric = evaluate.load("cer")

- 모델 예측을 입력으로 받아 CER 지표를 반환하는 함수인 compute_metrics를 정의한다.
- 이 함수는 먼저 label_ids에서 -100을 pad_token_id로 대체한다 (손실 계산 시 패딩된 토큰을 올바르게 무시하기 위해 데이터 콜레이터에서 적용한 단계를 되돌림).
- 그런 다음 예측된 id와 레이블 id를 문자열로 디코딩한다.
- 마지막으로 예측과 참조 레이블 사이의 CER을 계산한다

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install python-Levenshtein

  Obtaining dependency information for python-Levenshtein from https://files.pythonhosted.org/packages/ae/9c/208f8ad7eb38492ac4f829790a500bcfca88b1d0a1c988f6480a52a6f681/python_Levenshtein-0.21.1-py3-none-any.whl.metadata
  Obtaining dependency information for Levenshtein==0.21.1 from https://files.pythonhosted.org/packages/0b/66/0f66df5a5357aed9ff42684c571e590b86953fdf61cde1f5b9e26507ad10/Levenshtein-0.21.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.8/172.8 kB 2.7 MB/s eta 0:00:0012.9 MB/s eta 0:00:01


In [ ]:
import Levenshtein  # Levenshtein 패키지 설치 필요 (pip install python-Levenshtein)

# CER
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # label_ids에서 -100을 pad_token_id로 대체.
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # 예측된 토큰 id와 레이블 토큰 id를 문자열로 디코딩
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Character Error Rate (CER)를 계산
    cer = 100 * Levenshtein.distance("".join(pred_str), "".join(label_str)) / max(len("".join(pred_str)), len("".join(label_str)))

    return {"cer": cer}

### 사전 훈련된 Whisper ASR 모델의 체크포인트를 불러오기

Whisper base 사전 훈련된 체크포인트를 load하기.

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

### forced_decoder_ids는 디코더 출력을 강제로 지정하는 데 사용됨.
- 이를 사용하지 않으면 디코더는 자유롭게 다음 토큰을 선택하게 된다.

### suppress_tokens는 생성 과정에서 특정 토큰을 제거하는 데 사용됨.
- 이를 사용하지 않으면 모든 토큰이 생성에 사용될 수 있다.

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

# 훈련 구성 정의

훈련과 관련된 모든 매개변수를 정의.

In [ ]:
from transformers import Seq2SeqTrainingArguments
import accelerate
import transformers
import torch
training_args = Seq2SeqTrainingArguments(
    output_dir="./oceanstar/bridze",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",
    greater_is_better=False,
    push_to_hub=True,
    hub_private_repo = True,
    resume_from_checkpoint=True
)

In [ ]:
!pip install tensorboard

훈련 인자를 hugging face Trainer에 전달하여 모델을 훈련

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=bridze_data["train"],
    eval_dataset=bridze_data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/home/pmi-minos/Documents/whisper_tuning/./oceanstar/bridze is already a clone of https://huggingface.co/oceanstar/bridze. Make sure you pull the latest changes with `repo.git_pull()`.


훈련을 시작하기 전에 프로세서 객체를 한 번 저장.
프로세서는 훈련 중에 변경되지 않으므로 저장한 후 다시 사용할 수 있음.

In [ ]:
processor.save_pretrained(training_args.output_dir)

### Training

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

#train 시작
trainer.train("./oceanstar/bridze/checkpoint-1000")

Step,Training Loss,Validation Loss,Cer
1500,0.221300,0.238789,9.313154
2000,0.225700,0.219372,8.629522
2500,0.194100,0.206800,7.510904
3000,0.139500,0.196908,7.324709
3500,0.178700,0.190499,7.551738
4000,0.163900,0.188047,7.389370


TrainOutput(global_step=4000, training_loss=0.1541238793730736, metrics={'train_runtime': 41118.5048, 'train_samples_per_second': 0.778, 'train_steps_per_second': 0.097, 'total_flos': 2.5995886460928e+18, 'train_loss': 0.1541238793730736, 'epoch': 0.26})

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_13_0",
    "dataset": "bridzeDataset",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ko, split: test",
    "language": "ko",
    "model_name": "oceanstar-bridze",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-base",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

훈련 결과를 Hugging Face Hub에 업로드하고 프로세서 객체를 저장.

In [ ]:
trainer.push_to_hub(**kwargs)

Upload file runs/Aug03_09-29-30_pmiminos-Z390-AORUS-ELITE/events.out.tfevents.1691022582.pmiminos-Z390-AORUS-E…

To https://huggingface.co/oceanstar/bridze
   5ee65fd..c091d3d  main -> main

To https://huggingface.co/oceanstar/bridze
   c091d3d..e66a642  main -> main



'https://huggingface.co/oceanstar/bridze/commit/c091d3d28d777cb914de792610f6b275e0af2932'

## 데모생성

- 훈련된 모델을 사용하여 ASR 능력을 보여주는 데모를 구축해보기.
- hugging face Transformers의 pipeline을 사용하면 오디오 입력을 사전 처리하고 모델 예측을 디코딩하는 ASR 파이프라인 전체를 구현할 수 있음.
- 실행시 Gradio 데모가 생성되며, 이를 통해 컴퓨터의 마이크를 통해 음성을 녹음하고, 훈련된 Whisper 모델을 사용하여 해당 음성을 텍스트로 변환할 수 있다.

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="oceanstar/bridze")

def transcribe(audio):
    text = pipe(audio, chunk_length_s=30)["text"]
    return text


iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper base Korean",
    description="베트남 발화자 음성으로 fine tuning 된 데모",
)

iface.launch()

^C


Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


ImportError: cannot import name 'pipeline' from 'transformers' (c:\ProgramData\anaconda3\Lib\site-packages\transformers\__init__.py)